In [ ]:
from googleAPI import *

In [ ]:
with open("../../../apikey.txt", encoding="utf8") as f:
    apikey = f.readline().strip()

In [ ]:
get_youtube_api_json("search", {"key":apikey,"q":"공부", "part":"snippet", })

In [ ]:
get_youtube_api_json("videos", {"key":apikey, "part":"snippet", "chart":"mostPopular"})